# Pollen Metagenome of Fall Pollinators

Note that the sequence files and the qiime2 files containing all the sequences (demux-paired-end.qza) and the trimmed ITS sequences (trimmed.qza) are not provided here because of their size. The steps below allow you to recreate the files.

## Import sequences into Qiime.

The sequences for the 10 samples should be downloaded from GenBank's SRA database under Bioproject PRJNA854139.

Once downloaded the files should be moved to a directory called "Data" within the current directory.

In [ ]:
!qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path Data \
  --input-format CasavaOneEightSingleLanePerSampleDirFmt \
  --output-path 01-ProcessReads/demux-paired-end.qza

In [ ]:
!qiime demux summarize \
 --i-data 01-ProcessReads/demux-paired-end.qza \
 --p-n 100000 \
 --verbose \
 --o-visualization 01-ProcessReads/demux.qzv

## Isolate ITS2 sequences.

In [ ]:
!qiime itsxpress trim-pair-output-unmerged\
  --i-per-sample-sequences 01-ProcessReads/demux-paired-end.qza \
  --p-region ITS2 \
  --p-taxa ALL \
  --p-cluster-id 1.0 \
  --p-threads 30 \
  --o-trimmed 01-ProcessReads/trimmed.qza

In [ ]:
!qiime demux summarize \
 --i-data 01-ProcessReads/trimmed.qza \
 --p-n 100000 \
 --verbose \
 --o-visualization 01-ProcessReads/trimmed.qzv

## Identify ASVs

In [ ]:
!qiime dada2 denoise-paired \
  --i-demultiplexed-seqs 01-ProcessReads/trimmed.qza \
  --p-trunc-len-r 0 \
  --p-trunc-len-f 0 \
  --p-n-threads 30 \
  --o-representative-sequences 01-ProcessReads/rep-seqs-dada.qza \
  --o-table 01-ProcessReads/table-dada.qza \
  --o-denoising-stats 01-ProcessReads/stats-dada.qza

In [ ]:
!qiime metadata tabulate \
  --m-input-file 01-ProcessReads/stats-dada.qza \
  --o-visualization 01-ProcessReads/stats-dada.qzv

## Determine taxonomy

In [ ]:
!qiime feature-classifier classify-sklearn \
  --i-classifier 01-ProcessReads/ITS2_classifier.qza \
  --i-reads 01-ProcessReads/rep-seqs-dada.qza \
  --p-n-jobs 30 \
  --o-classification 01-ProcessReads/taxonomydev.qza

In [ ]:
!qiime metadata tabulate \
  --m-input-file 01-ProcessReads/taxonomydev.qza \
  --o-visualization 01-ProcessReads/taxonomydev.qzv

## Make Phylogenetic Tree

In [ ]:
!qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences 01-ProcessReads/rep-seqs-dada.qza \
  --o-alignment 01-ProcessReads/aligned-rep-seqs.qza \
  --o-masked-alignment 01-ProcessReads/masked-aligned-rep-seqs.qza \
  --o-tree 01-ProcessReads/unrooted-tree.qza \
  --o-rooted-tree 01-ProcessReads/rooted-tree.qza

## Analyses

In [ ]:
!qiime diversity core-metrics-phylogenetic \
  --i-phylogeny 01-ProcessReads/rooted-tree.qza \
  --i-table 01-ProcessReads/table-dada.qza \
  --p-sampling-depth 12000 \
  --m-metadata-file 021022ECits2S2F-mapping.txt \
  --output-dir 02-Results 

In [ ]:
# Taxa plots
!qiime taxa barplot \
  --i-table 01-ProcessReads/table-dada.qza \
  --i-taxonomy 01-ProcessReads/taxonomydev.qza \
  --m-metadata-file 021022ECits2S2F-mapping.txt \
  --o-visualization 02-Results/taxa-bar-plots-vsearch.qzv

# Rarefaction Plots
!qiime diversity alpha-rarefaction \
  --i-table 01-ProcessReads/table-dada.qza \
  --i-phylogeny 01-ProcessReads/rooted-tree.qza \
  --p-max-depth 12000 \
  --m-metadata-file 021022ECits2S2F-mapping.txt \
  --o-visualization 02-Results/alpha-rarefaction.qzv

